<a href="https://colab.research.google.com/github/pranavsrinivas29/hyperparameter_opt/blob/main/Regression_SVR%2CXG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Regressors - SVR, XG

Hyp-opt - Raandom,Grid Based, Bayesian,Gradient-Based(L-BFGS-B, CG, POWELL)

In [4]:
import numpy as np
import pandas as pd
import warnings
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [5]:


# Load the Diabetes dataset
data = load_diabetes()
X = data.data
y = data.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create separate pipelines for SVR and XGBoost
svr_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('regressor', SVR(kernel='linear'))  # Support Vector Machine for Regression
])

xgboost_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('regressor', XGBRegressor())  # XGBoost for Regression
])

# Fit the SVR pipeline on the training data
svr_pipeline.fit(X_train, y_train)

# Predict using SVR
y_svr_pred = svr_pipeline.predict(X_test)

# Fit the XGBoost pipeline on the training data
xgboost_pipeline.fit(X_train, y_train)

# Predict using XGBoost
y_xgboost_pred = xgboost_pipeline.predict(X_test)

# Calculate Mean Squared Error (MSE) for SVR
mse_svr = mean_squared_error(y_test, y_svr_pred)
print(f"SVR Mean Squared Error (MSE): {mse_svr:.4f}")

# Calculate Mean Squared Error (MSE) for XGBoost
mse_xgboost = mean_squared_error(y_test, y_xgboost_pred)
print(f"XGBoost Mean Squared Error (MSE): {mse_xgboost:.4f}")


SVR Mean Squared Error (MSE): 2856.7597
XGBoost Mean Squared Error (MSE): 3513.6592


Random Search

In [20]:
# Define parameter spaces for SVR and XGBoost
param_space_svr = {
    'regressor__C': np.logspace(0.1, 1, 10),  # Regularization parameter for SVR
    'regressor__epsilon': np.logspace(-3, 1, 100)  # Epsilon parameter for SVR
}

param_space_xgboost = {
    'regressor__learning_rate': [0.01, 0.1, 0.25],  # Learning rate for XGBoost
    'regressor__n_estimators': [50, 100, 250],  # Number of boosting rounds
    'regressor__max_depth':  [3, 4, 5],  # Maximum depth of trees
    'regressor__min_child_weight': [1, 2, 3],  # Minimum sum of instance weight in a child
    'regressor__subsample': [0.7, 0.8, 1.0],  # Row subsample ratio of training instances
    'regressor__colsample_bytree': [0.7, 0.8, 1.0]  # Column subsample ratio of features
}



In [21]:
# Perform Randomized Search for SVR hyperparameters
svr_random_search = RandomizedSearchCV(
    svr_pipeline,
    param_space_svr,
    n_iter=50,  # Number of random combinations to try
    scoring='neg_mean_squared_error',  # Use negative MSE for scoring
    cv=5,  # Number of cross-validation folds
    random_state=42,
    n_jobs=-1  # Use all available CPU cores for parallelization
)

# Fit SVR Randomized Search on the training data
svr_random_search.fit(X_train, y_train)

# Get the best SVR model
best_svr_model = svr_random_search.best_estimator_

# Perform Randomized Search for XGBoost hyperparameters
xgboost_random_search = RandomizedSearchCV(
    xgboost_pipeline,
    param_space_xgboost,
    n_iter=50,  # Number of random combinations to try
    scoring='neg_mean_squared_error',  # Use negative MSE for scoring
    cv=5,  # Number of cross-validation folds
    random_state=42,
    n_jobs=-1  # Use all available CPU cores for parallelization
)

# Fit XGBoost Randomized Search on the training data
xgboost_random_search.fit(X_train, y_train)

# Get the best XGBoost model
best_xgboost_model = xgboost_random_search.best_estimator_

# Predict using the best SVR model
y_svr_pred = best_svr_model.predict(X_test)

# Predict using the best XGBoost model
y_xgboost_pred = best_xgboost_model.predict(X_test)


In [22]:
# Calculate Mean Squared Error (MSE) for SVR
mse_svr = mean_squared_error(y_test, y_svr_pred)
print(f"SVR Mean Squared Error (MSE): {mse_svr:.4f}")

# Calculate Mean Squared Error (MSE) for XGBoost
mse_xgboost = mean_squared_error(y_test, y_xgboost_pred)
print(f"XGBoost Mean Squared Error (MSE): {mse_xgboost:.4f}")

# Print the best hyperparameters for SVR and XGBoost
print("Best SVR Hyperparameters:")
print(svr_random_search.best_params_)

print("\nBest XGBoost Hyperparameters:")
print(xgboost_random_search.best_params_)

SVR Mean Squared Error (MSE): 2829.0397
XGBoost Mean Squared Error (MSE): 2795.3419
Best SVR Hyperparameters:
{'regressor__epsilon': 10.0, 'regressor__C': 1.9952623149688797}

Best XGBoost Hyperparameters:
{'regressor__subsample': 0.7, 'regressor__n_estimators': 250, 'regressor__min_child_weight': 3, 'regressor__max_depth': 5, 'regressor__learning_rate': 0.01, 'regressor__colsample_bytree': 0.7}


Grid Based

In [25]:
# Perform Grid Search for SVR hyperparameters
svr_grid_search = GridSearchCV(
    svr_pipeline,
    param_space_svr,
    scoring='neg_mean_squared_error',  # Use negative MSE for scoring
    cv=5  # Number of cross-validation folds
)

# Fit SVR Grid Search on the training data
svr_grid_search.fit(X_train, y_train)

# Get the best SVR model
best_svr_model = svr_grid_search.best_estimator_

# Perform Grid Search for XGBoost hyperparameters
xgboost_grid_search = GridSearchCV(
    xgboost_pipeline,
    param_space_xgboost,
    scoring='neg_mean_squared_error',  # Use negative MSE for scoring
    cv=5  # Number of cross-validation folds
)

# Fit XGBoost Grid Search on the training data
xgboost_grid_search.fit(X_train, y_train)

# Get the best XGBoost model
best_xgboost_model = xgboost_grid_search.best_estimator_

# Predict using the best SVR model
y_svr_pred = best_svr_model.predict(X_test)

# Predict using the best XGBoost model
y_xgboost_pred = best_xgboost_model.predict(X_test)

In [26]:
# Calculate Mean Squared Error (MSE) for SVR
mse_svr = mean_squared_error(y_test, y_svr_pred)
print(f"SVR Mean Squared Error (MSE): {mse_svr:.4f}")

# Calculate Mean Squared Error (MSE) for XGBoost
mse_xgboost = mean_squared_error(y_test, y_xgboost_pred)
print(f"XGBoost Mean Squared Error (MSE): {mse_xgboost:.4f}")

# Print the best hyperparameters for SVR and XGBoost
print("Best SVR Hyperparameters:")
print(svr_grid_search.best_params_)

print("\nBest XGBoost Hyperparameters:")
print(xgboost_grid_search.best_params_)

SVR Mean Squared Error (MSE): 2854.1568
XGBoost Mean Squared Error (MSE): 2756.8413
Best SVR Hyperparameters:
{'regressor__C': 1.5848931924611136, 'regressor__epsilon': 10.0}

Best XGBoost Hyperparameters:
{'regressor__colsample_bytree': 0.7, 'regressor__learning_rate': 0.01, 'regressor__max_depth': 3, 'regressor__min_child_weight': 3, 'regressor__n_estimators': 250, 'regressor__subsample': 0.8}


In [29]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.3 MB/s eta 0:00:00


In [30]:
 from skopt import BayesSearchCV

In [31]:
# Perform Bayesian optimization for SVR hyperparameters
svr_bayesian_search = BayesSearchCV(
    svr_pipeline,
    param_space_svr,
    n_iter=50,  # Number of random combinations to try
    scoring='neg_mean_squared_error',  # Use negative MSE for scoring
    cv=5,  # Number of cross-validation folds
    random_state=42,
    n_jobs=-1  # Use all available CPU cores for parallelization
)

# Fit SVR Bayesian Search on the training data
svr_bayesian_search.fit(X_train, y_train)

# Get the best SVR model
best_svr_model = svr_bayesian_search.best_estimator_

# Perform Bayesian optimization for XGBoost hyperparameters
xgboost_bayesian_search = BayesSearchCV(
    xgboost_pipeline,
    param_space_xgboost,
    n_iter=50,  # Number of random combinations to try
    scoring='neg_mean_squared_error',  # Use negative MSE for scoring
    cv=5,  # Number of cross-validation folds
    random_state=42,
    n_jobs=-1  # Use all available CPU cores for parallelization
)

# Fit XGBoost Bayesian Search on the training data
xgboost_bayesian_search.fit(X_train, y_train)

# Get the best XGBoost model
best_xgboost_model = xgboost_bayesian_search.best_estimator_

# Predict using the best SVR model
y_svr_pred = best_svr_model.predict(X_test)

# Predict using the best XGBoost model
y_xgboost_pred = best_xgboost_model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist

In [32]:
print(f"SVR Mean Squared Error (MSE): {mse_svr:.4f}")

# Calculate Mean Squared Error (MSE) for XGBoost
mse_xgboost = mean_squared_error(y_test, y_xgboost_pred)
print(f"XGBoost Mean Squared Error (MSE): {mse_xgboost:.4f}")

# Print the best hyperparameters for SVR and XGBoost
print("Best SVR Hyperparameters:")
print(svr_bayesian_search.best_params_)

print("\nBest XGBoost Hyperparameters:")
print(xgboost_bayesian_search.best_params_)

SVR Mean Squared Error (MSE): 2854.1568
XGBoost Mean Squared Error (MSE): 2756.8413
Best SVR Hyperparameters:
OrderedDict([('regressor__C', 3.9810717055349722), ('regressor__epsilon', 2.0565123083486534)])

Best XGBoost Hyperparameters:
OrderedDict([('regressor__colsample_bytree', 0.7), ('regressor__learning_rate', 0.01), ('regressor__max_depth', 3), ('regressor__min_child_weight', 3), ('regressor__n_estimators', 250), ('regressor__subsample', 0.8)])


In [6]:
from scipy.optimize import minimize

In [18]:
def optimize_xgboost_hyperparameters(params):
    # Extract hyperparameters from the flattened list
    learning_rate, n_estimators, max_depth, min_child_weight, subsample, colsample_bytree = params

    learning_rate = max(learning_rate, 0)
    min_child_weight = max(min_child_weight, 0)
    subsample = max(0, min(subsample, 1))
    colsample_bytree= max(0, min(colsample_bytree, 1))
    # Round n_estimators and convert max_depth to an integer
    n_estimators = int(round(n_estimators))
    max_depth = int(max_depth)

    xgboost_model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree
    )

    xgboost_model.fit(X_train, y_train)
    y_pred = xgboost_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Initial hyperparameters as a flattened list
initial_xgboost_hyperparameters = [0.1, 100.0, 3.0, 1.0, 0.8, 0.8]

# Perform hyperparameter optimization using minimize
result_xgboost = minimize(optimize_xgboost_hyperparameters, initial_xgboost_hyperparameters, method='L-BFGS-B')



In [19]:
# Get the best hyperparameters
best_xgboost_hyperparameters = result_xgboost.x
best_mse = result_xgboost.fun  # Best Mean Squared Error

print("Best XGBoost Hyperparameters:")
print(best_xgboost_hyperparameters)
print("Best MSE:", best_mse)

Best XGBoost Hyperparameters:
[7.4859006e-02 1.0000000e+02 3.0000000e+00 1.0000000e+00 8.0000000e-01
 8.0000000e-01]
Best MSE: 2845.0603979640064


In [24]:
# Define a custom objective function for hyperparameter optimization
def optimize_svm_hyperparameters(params):
    # Extract hyperparameters from the flattened list
    C, epsilon = params
    epsilon = max(epsilon, 0.0)
    C = np.exp(C)
    svm_model = SVR(
        kernel='linear',  # You can modify the kernel type as needed
        C=C,
        epsilon=epsilon
    )

    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse

In [25]:
# Initial hyperparameters as a flattened list
initial_svm_hyperparameters = [1.0, 1e-3]  # You can modify these values as needed

# Perform hyperparameter optimization using minimize
result_svm = minimize(optimize_svm_hyperparameters, initial_svm_hyperparameters, method='L-BFGS-B')

# Get the best hyperparameters
best_svm_hyperparameters = result_svm.x
best_mse_svm = result_svm.fun  # Best Mean Squared Error

print("Best SVM Hyperparameters:")
print("C:", best_svm_hyperparameters[0])
print("Epsilon:", best_svm_hyperparameters[1])
print("Best MSE:", best_mse_svm)

Best SVM Hyperparameters:
C: 5.698712502732738
Epsilon: -1.123426785736673
Best MSE: 2783.263459906919


In [26]:
result_svm2 = minimize(optimize_svm_hyperparameters, initial_svm_hyperparameters, method='POWELL')
result_svm3 = minimize(optimize_svm_hyperparameters, initial_svm_hyperparameters, method='CG')

In [29]:
# Get the best hyperparameters
best_svm_hyperparameters2 = result_svm2.x
best_mse_svm2 = result_svm2.fun  # Best Mean Squared Error


# Get the best hyperparameters
best_svm_hyperparameters3 = result_svm3.x
best_mse_svm3 = result_svm3.fun  # Best Mean Squared Error

In [32]:
print("Best SVM Hyperparameters for POWELL:")
print("C:", best_svm_hyperparameters2[0])
print("Epsilon:", best_svm_hyperparameters2[1])
print("Best MSE:", best_mse_svm2)

print("")
print("Best SVM Hyperparameters for CG:")
print("C:", best_svm_hyperparameters3[0])
print("Epsilon:", best_svm_hyperparameters3[1])
print("Best MSE:", best_mse_svm3)


Best SVM Hyperparameters for POWELL:
C: 5.698722085989913
Epsilon: -4.186342653703974
Best MSE: 2783.2630497056343

Best SVM Hyperparameters for CG:
C: 4.916209106182921
Epsilon: -0.0004355729757453898
Best MSE: 2892.4779961621766
